In [1]:
from util import *

from sklearn.metrics import confusion_matrix, recall_score, f1_score

In [2]:
dname = 'madelon'

In [3]:
ds = read_libsvm(dname)

clf = cgb(verbose=False)
clf.fit(ds[0], ds[1])
clf.score(ds[2], ds[3])

0.8533333333333334

In [4]:
# shadow dataset

x1 = [x for x in ds[0]]
x2 = [x for x in ds[2]]
xs = x1 + x2
nf = len(xs[0])
for x in xs:
    for i in range(nf):
        x[i] += np.random.laplace(0, 100)

nn = len(xs) >> 1

ys = [clf.predict(x) for x in xs]

xt = xs[nn:]
yt = xs[nn:]

xs = xs[:nn]
ys = ys[:nn]

In [5]:
# shadow model & adv dataset

nm = 16
n = len(xs)
j = 0
l = n // nm

xa = []
ya = []

sms = [cgb(verbose=False) for _ in range(nm)]
for i in range(nm):
    sms[i].fit(xs[j:j+l], ys[j:j+l])
    for x in xs[j:j+l]:
        xa.append(sms[i].predict_proba(x))
        ya.append(1)
    j += l

for x in xt:
    md = random.choice(sms)
    xa.append(md.predict_proba(x))
    ya.append(0)

In [6]:
# adv model
amd = cgb(verbose=False)
amd.fit(xa, ya)

In [12]:
ds = read_libsvm(dname)

xxs = [x for x in ds[0]]
xxs = xxs[:len(ds[2])]
yys = [1] * len(xxs)

xxs += [x for x in ds[2]]
yys += [0] * len(ds[2])

dd = ds[0][:500]
for x in dd:
    for i in range(nf):
        x[i] += np.random.laplace(0, 100)

xxs += dd.tolist()
yys += [0] * len(dd)

In [13]:
def infer(md, tar, x):
    p = tar.predict_proba(x)
    return md.predict(p)

In [14]:
pred = [infer(amd, clf, x) for x in xxs]

In [15]:
mt = confusion_matrix(yys, pred)
tp = mt[1][1]
fp = mt[0][1]
r = tp / fp
r

0.44680851063829785

In [16]:
accuracy_score(yys, pred) *100, roc_auc_score(yys, pred), f1_score(yys, pred), recall_score(yys, pred)

(55.529411764705884, 0.47681818181818186, 0.25, 0.21)